<a href="https://colab.research.google.com/github/lapatradaa/M-MMT4NL/blob/main/s_bert_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai sentence-transformers pandas tqdm


In [2]:
import openai
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

In [3]:
INPUT_CSV = "/content/s-bert - original.csv"         # your input file (must have column 'original')
OUTPUT_CSV = "results.csv"
SBERT_THRESHOLD = 0.75

#perturb

In [1]:
from openai import OpenAI


##fairness

In [10]:
PROMPT_FAIRNESS= """You are an expert bilingual linguist proficient in English and Thai.
Add one gender, race, or nationality descriptor to a given Thai or English sentence while preserving the sentiment and meaning.
Context:
 This task introduces demographic attributes to evaluate fairness and bias in model behavior.
 Add only one natural demographic detail (e.g., gender, nationality, ethnicity) without changing tone or grammatical correctness.

The output must remain grammatically and semantically coherent. Do not mix between languages.
The output must remain entirely in Thai and should sound natural to a native speaker.


References (Examples):
English:
“I’m so tired” to “She is so tired”


“I’m enjoying this relaxing day at home” to “I’m enjoying this relaxing day at Thai home”


“I love going on adventures” to “I love going on adventures with Japanese friends”


Thai:
“ฉันเหนื่อยมาก” to “เธอเหนื่อยมาก”


“ฉันชอบเดินทางท่องเที่ยว” to “ฉันชอบเดินทางท่องเที่ยวกับเพื่อนชาวญี่ปุ่น”


“เขาเป็นคนใจดี” to “เขาเป็นผู้ชายใจดี”


Thai sentence: "{}"
"""


def gpt_fairness(sentence: str) -> str:
    prompt = PROMPT_FAIRNESS.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            fairness = gpt_fairness(s)
            score = sbert_similarity(model, s, fairness)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "fairness": fairness,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "fairness": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)




Processing: 100%|██████████| 50/50 [00:57<00:00,  1.15s/it]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##temporal

In [ ]:
PROMPT_TEMPORAL = """You are a text perturbation assistant specializing in adding **temporal context** to Thai or English sentences.
Your goal is to contrast *past uncertainty* with *present certainty or state* by prepending a short temporal phrase.

### Objective
Transform the input sentence by adding a brief temporal prefix that contrasts the past (uncertainty, ambiguity, or change) with the present (current feeling, belief, or situation).

### Transformation Pattern
- Prepend a temporal phrase (e.g., “Not sure how it was before but”, “I didn’t feel this way before but”, or a Thai equivalent).
- Keep the rest of the sentence **exactly the same**.
- Maintain the original tone, meaning, and sentiment.
- Create a clear sense of temporal contrast.

### Examples (English)
- “I’m so tired” → “Not sure how it was before but I’m so tired”
- “I’m really hungry” → “Not sure how it was before but I’m really hungry”
- “I’m not sure if I can make it to the event” → “Not sure how it was before but I’m not sure if I can make it to the event”
- “It feels different today” → “Didn’t notice it before but it feels different today”

### Examples (Thai)
- “ฉันรู้สึกดีขึ้น” → “ไม่แน่ใจว่าเมื่อก่อนเป็นยังไง แต่ฉันรู้สึกดีขึ้น”
- “วันนี้เหนื่อยมาก” → “ไม่แน่ใจว่าเมื่อก่อนเป็นยังไง แต่วันนี้เหนื่อยมาก”
- “ฉันไม่แน่ใจว่าจะไปได้ไหม” → “ไม่แน่ใจว่าเมื่อก่อนเป็นยังไง แต่ฉันไม่แน่ใจว่าจะไปได้ไหม”

### Rules
1. Keep the original sentence intact after the temporal phrase.
2. Add a temporal contrast implying change or uncertainty in the past.
3. Preserve tone, sentiment, and style of the input.
4. Return only the transformed sentence — no explanations.

Thai or English sentence: "{}"
"""


def gpt_temporal(sentence: str) -> str:
    prompt = PROMPT_TEMPORAL.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            temporal = gpt_temporal(s)
            score = sbert_similarity(model, s, temporal)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "temporal": temporal,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "temporal": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)




Processing: 100%|██████████| 50/50 [00:54<00:00,  1.09s/it]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##SRL

In [11]:
PROMPT_SRL = """SRL
You are an expert bilingual linguist proficient in English and Thai.
Your task is to convert a question written in active voice into its passive voice equivalent, using semantic role labeling (SRL) principles to ensure correct subject–object role inversion and meaning preservation.
Context:
 This task is to accurately transform syntactic structures while maintaining semantics.
 In active voice, the subject performs the action; in passive voice, the subject receives the action.
 When converting, you must correctly identify the agent (doer) and patient (receiver) in the sentence, then reassign their grammatical positions appropriately.
 The question format must remain intact.
The output must remain grammatically and semantically coherent. Do not mix between languages.
The output must remain entirely in Thai and should sound natural to a native speaker.


References (Examples):
 English:
“Are you likely to find a crucifix in Karachi?” to “Is a crucifix in Karachi likely to be found by you?”


“Could the main character of Alice’s Adventures in Wonderland join a Masonic Lodge?” to “Could the Masonic Lodge be joined by the main character of Alice’s Adventures in Wonderland?”


“Is Romeo and Juliet an unusual title to teach high schoolers?” to “Would Romeo and Juliet be considered an unusual title to be taught to high schoolers?”


Thai:
“เขาอ่านหนังสือเล่มนี้หรือไม่?” to “หนังสือเล่มนี้ถูกอ่านหรือไม่?”


“นักเรียนจะเขียนรายงานนี้ไหม?” to “รายงานนี้จะถูกเขียนไหม?”


“พวกเขาจะสร้างบ้านหลังนี้ไหม?” to “บ้านหลังนี้จะถูกสร้างไหม?”
Thai sentence: "{}"
"""




def gpt_srl(sentence: str) -> str:
    prompt = PROMPT_SRL.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            srl = gpt_srl(s)
            score = sbert_similarity(model, s, srl)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "srl": srl,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "srl": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)




Processing: 100%|██████████| 50/50 [00:49<00:00,  1.00it/s]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##coference

In [ ]:
PROMPT_COREFERENCE = """You're an expert linguist in English and Thai.
Your task is to modify this **Thai or English** sentence by adding a coreference resolution pattern in the **same language as the input**.

You are a text perturbation assistant that converts questions or statements by explicitly resolving coreferences — making references (like "he", "she", "it", "they", or Thai equivalents) clear.

---

OBJECTIVE
- Add a phrase like “Considering [entity/subject], …” in English **or** “เมื่อพิจารณา[entity/subject], …” in Thai.
- Maintain the original question or statement format.
- Use correct pronouns or referents in context.
- Do **not** mix languages. If the input is Thai, output must be fully Thai.

---

English Examples
- "Are you likely to find a crucifix in Karachi?" → "Considering yourself, are you likely to find a crucifix in Karachi?"
- "Is Romeo and Juliet an unusual title to teach high schoolers?" → "Considering Romeo and Juliet, is it an unusual title to teach high schoolers?"

Thai Examples
- "งานนี้ท้าทายแต่สนุก" → "เมื่อพิจารณางานนี้แล้ว มันท้าทายแต่สนุก"
- "กาแฟแก้วนี้ช่วยให้ฉันตื่นไหม?" → "เมื่อพิจารณากาแฟแก้วนี้แล้ว มันช่วยให้ฉันตื่นไหม?"
- "พวกเขาพึงพอใจกับผลการสอบ" → "เมื่อพิจารณาพวกเขาแล้ว พวกเขาพึงพอใจกับผลการสอบ"
- "บทเรียนนี้ยากไหม?" → "เมื่อพิจารณาบทเรียนนี้แล้ว มันยากไหม?"
- "เขาตื่นเต้นกับโอกาสนี้ไหม?" → "เมื่อพิจารณาเขาแล้ว เขาตื่นเต้นกับโอกาสนี้ไหม?"

---

RULES
1. Preserve the **language of the input** — Thai in, Thai out; English in, English out.
2. Add the prefix:
   - English: “Considering [entity],”
   - Thai: “เมื่อพิจารณา[entity]แล้ว”
3. Keep the rest of the sentence intact.
4. Resolve pronouns explicitly if necessary (e.g., replace “เขา” with “เขาเอง” if it clarifies).
5. Do not translate or mix languages.
6. Return only the transformed sentence — no explanations.

---

Thai sentence: "{}"
"""


def gpt_coreference(sentence: str) -> str:
    prompt = PROMPT_COREFERENCE.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            coreference = gpt_coreference(s)
            score = sbert_similarity(model, s, coreference)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "coreference": coreference,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "coreference": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)




Processing: 100%|██████████| 50/50 [00:46<00:00,  1.07it/s]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##vocab

In [9]:
PROMPT_VOCAB = """You are an expert bilingual linguist proficient in English and Thai.
Your task is to add exactly one word to the given Thai sentence that slightly enriches or clarifies the meaning without changing its overall semantics or tone.
Context:
 This exercise focuses on vocabulary enhancement while maintaining naturalness and meaning.
 The additional word should fit smoothly into the sentence, adding nuance, specificity, or mild emphasis — not altering the intent.
 Avoid inserting random intensifiers or unrelated words.
The output must remain grammatically and semantically coherent. Do not mix between languages.
The output must remain entirely in Thai and should sound natural to a native speaker.


References (Examples):
 English examples of this modification:
“I’m so tired” to “I’m so really tired”


“I’m not sure what to do with my weekend” to “I’m not sure what to do with my available weekend”
“I’m so stressed out about my financial situation” to “I’m so stressed out about my current financial situation”


Thai examples:
“ฉันเหนื่อยมาก” to “ฉันเหนื่อยมากจริงๆ”


“ฉันชอบวันหยุดนี้” to “ฉันชอบวันหยุดพิเศษนี้”


“เขาทำงานหนัก” to “เขาทำงานหนักมาก”


Thai sentence: "{}"
"""


def gpt_vocab(sentence: str) -> str:
    prompt = PROMPT_VOCAB.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            vocab = gpt_vocab(s)
            score = sbert_similarity(model, s, vocab)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "vocab": vocab,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "vocab": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)

Processing: 100%|██████████| 50/50 [00:43<00:00,  1.15it/s]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##negation

In [7]:
PROMPT_NEGATION = """You are an expert bilingual linguist proficient in English and Thai.
Your task is to negate the given Thai sentence while preserving its original semantic meaning and grammatical naturalness.
Context:
Negation involves transforming a statement into a double negative form without changing its core meaning.
The output must remain grammatically and semantically coherent. Do not mix between languages.
The output must remain entirely in Thai and should sound natural to a native speaker.
Avoid literal translations of “not” — instead, use natural Thai negation patterns such as “ไม่”, “ไม่ได้”, or “ไม่มี” depending on the sentence structure.
References (Examples):
 English examples of similar modifications:
“I’m so tired” to “I’m so not energetic”


“I’m really hungry” to “I’m really not full”


“I’m not sure if I can make it to the event” to “I’m unsure if I can make it to the event”


Thai examples could be:
“ฉันเหนื่อยมาก” to “ฉันไม่ค่อยมีพลัง”


“วันนี้อากาศดี” to “วันนี้อากาศไม่แย่”


“เธอเข้าใจบทเรียนนี้ดี” to “เธอไม่สับสนกับบทเรียนนี้”
Thai sentence: "{}"


"""


def gpt_negation(sentence: str) -> str:
    prompt = PROMPT_NEGATION.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())

if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            negated = gpt_negation(s)
            score = sbert_similarity(model, s, negated)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "negated": negated,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "negated": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)

Processing: 100%|██████████| 50/50 [00:49<00:00,  1.01it/s]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##taxonomy

In [6]:
PROMPT_TAXONOMY = """You are an expert linguist in English and Thai.
Your job is to modify a given Thai sentence by replacing one word with its synonym, while keeping the full semantic meaning and natural tone intact.

Context:

This task focuses on lexical variation in Thai. The goal is to show how synonym substitution can maintain meaning while providing diversity in expression.

The output must remain grammatically and semantically coherent. Do not mix between languages. The output must remain entirely in Thai and should sound natural to a native speaker.

References (Examples):
English examples of similar modifications:
“I’m so tired” to “I’m so exhausted”


“I’m really hungry” to “I’m really starving”


“I’m not sure if I can make it to the event” to “I’m not confident if I can make it to the event”


Thai examples could be:
“ฉันชื่นชมผลงานศิลปะนี้มาก” to “ฉันชื่นชอบผลงานศิลปะนี้มาก”


“วันนี้อากาศดีและสดชื่น” to “วันนี้อากาศดีและแจ่มใส”


“ฉันเบื่อชีวิตประจำวันแบบนี้” to “ฉันเซ็งชีวิตประจำวันแบบนี้”



Thai sentence: "{}"

"""

def gpt_taxonomy(sentence: str) -> str:
    prompt = PROMPT_TAXONOMY.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            taxonomy = gpt_taxonomy(s)
            score = sbert_similarity(model, s, taxonomy)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "taxonomy": taxonomy,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "taxonomy": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)

Processing: 100%|██████████| 50/50 [02:30<00:00,  3.01s/it]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##NER

In [8]:
PROMPT_NER = """You are an expert bilingual linguist proficient in English and Thai.
Your task is to replace the named entity (e.g., person’s name or pronoun) in the given Thai sentence with a different proper name, while keeping the sentence’s meaning and structure natural.
Context:
The goal is to perform a named entity replacement — not translation or paraphrasing.
You should change the reference (such as replacing “ฉัน” or “เขา” with “เจน”, “แจ็ค”, etc.) while ensuring that the sentence remains fluent and semantically intact.
Do not alter verb tense, sentence tone, or polarity.
The output must remain grammatically and semantically coherent. Do not mix between languages.
The output must remain entirely in Thai and should sound natural to a native speaker.


References (Examples):
 English examples of the same modification:
“I’m so tired” to “Jane is so tired”


“I’m really hungry” to “Jack is really hungry”


“I’m not sure if I can make it to the event” to “Jill is not sure if she can make it to the event”


Thai examples:
“ฉันเหนื่อยมาก” to “เจนเหนื่อยมาก”


“ผมหิวมาก” to “แจ็คหิวมาก”


“ฉันไม่แน่ใจว่าจะไปได้ไหม” to “จิลไม่แน่ใจว่าเธอจะไปได้ไหม”


Thai sentence: "{}"
"""


def gpt_ner(sentence: str) -> str:
    prompt = PROMPT_NER.format(sentence)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    result = response.choices[0].message.content.strip()
    # remove leading/trailing quotes if present
    if result.startswith('"') and result.endswith('"'):
        result = result[1:-1]
    if result.startswith("'") and result.endswith("'"):
        result = result[1:-1]
    return result.strip()


def sbert_similarity(model, sent1, sent2):
    emb1 = model.encode(sent1, convert_to_tensor=True)
    emb2 = model.encode(sent2, convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2)
    return float(score.item())


if __name__ == "__main__":
    df = pd.read_csv(INPUT_CSV)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    results = []
    for s in tqdm(df['original'], desc="Processing"):
        try:
            ner = gpt_ner(s)
            score = sbert_similarity(model, s, ner)
            passed = "✅ Pass" if score >= SBERT_THRESHOLD else "❌ Fail"
            results.append({
                "original": s,
                "ner": ner,
                "sbert_score": round(score, 4),
                "result": passed
            })
        except Exception as e:
            results.append({
                "original": s,
                "ner": None,
                "sbert_score": None,
                "result": f"Error: {e}"
            })

    out_df = pd.DataFrame(results)
    out_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')
    print(out_df)

Processing: 100%|██████████| 50/50 [00:48<00:00,  1.02it/s]

                                       original  \
0             ฉันมีความสุขมากที่ได้พบเพื่อนใหม่   
1          ฉันตื่นเต้นกับการเดินทางไปต่างประเทศ   
2                  ฉันไม่พอใจกับบริการที่ได้รับ   
3                     ฉันรู้สึกเหนื่อยและหมดแรง   
4                      เพลงนี้ทำให้ฉันรู้สึกสงบ   
5                     ฉันไม่ชอบกลิ่นอาหารจานนี้   
6               ฉันรู้สึกมั่นใจในทักษะของตัวเอง   
7                         งานนี้ท้าทายแต่ก็สนุก   
8                      พวกเขาพึงพอใจกับผลการสอบ   
9                  กาแฟแก้วนี้ช่วยให้ฉันตื่นตัว   
10           บรรยากาศร้านนี้เงียบสงบและผ่อนคลาย   
11              บทเรียนนี้ยาก แต่เธอตั้งใจเรียน   
12            หนังสือเล่มนี้ให้แรงบันดาลใจดีมาก   
13                        อาหารมื้อนี้อร่อยสุดๆ   
14                  เขาตื่นเต้นกับโอกาสครั้งนี้   
15             การเดินทางครั้งนี้สนุกและน่าจดจำ   
16     ฉันรู้สึกอบอุ่นใจเมื่อได้อยู่กับครอบครัว   
17                 ฉันผิดหวังกับคุณภาพสินค้านี้   
18            เขาช่วยฉันแก้ปัญห

##robustness

In [ ]:
import random
import string
import math

class Perturbation:
    """Lightweight offline version of the Perturbation class."""

    def robust(self, text, num=20):
        """
        Create multiple sentences where each sentence has exactly one different word perturbed.
        """
        words = text.split()
        perturbed_sentences = []
        eligible_words = [i for i, word in enumerate(words) if len(word.rstrip(string.punctuation)) >= 2]

        if not eligible_words:
            return [text]

        num = max(1, min(100, num))
        words_to_swap = math.ceil(len(eligible_words) * (num / 100.0))
        words_to_swap = min(words_to_swap, len(eligible_words))
        indices_to_swap = random.sample(eligible_words, words_to_swap)

        for i in indices_to_swap:
            modified = words.copy()
            w = words[i]
            if len(w) > 2:
                j = random.randint(0, len(w) - 2)
                perturbed = w[:j] + w[j+1] + w[j] + w[j+2:]
                modified[i] = perturbed
                perturbed_sentences.append(" ".join(modified))
        return perturbed_sentences


In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # handles Thai

p = Perturbation()

def calculate_robustness(text, model, p, num=20):
    perturbed_sentences = p.robust(text, num)
    original_emb = model.encode(text, convert_to_tensor=True)
    pert_embs = model.encode(perturbed_sentences, convert_to_tensor=True)
    sims = [float(util.cos_sim(original_emb, emb)) for emb in pert_embs]
    robustness_score = np.mean(sims)
    return robustness_score, perturbed_sentences


In [ ]:
# Load your Thai sentences
df = pd.read_csv("/content/s-bert - original.csv")

scores, perturbations = [], []
for text in df["original"]:
    score, pert = calculate_robustness(str(text), model, p)
    scores.append(score)
    perturbations.append(pert)

df["robustness"] = scores
df["perturbations"] = perturbations

# 🧹 Clean up [' '] brackets and quotes
df["perturbations"] = df["perturbations"].apply(lambda x: ", ".join(x) if isinstance(x, list) else str(x))

df.to_excel("robustness_results.xlsx", index=False)
print("✅ Saved to robustness_results.xlsx (clean version)")



✅ Saved to robustness_results.xlsx (clean version)
